In [ ]:
import pandas as pd
import jupyter_black
from g_nfl.utils.database import PicksDatabase
from g_nfl.utils.database import PoolSpreadsDatabase
from g_nfl.utils.data import get_current_nfl_week
import nfl_data_py as nfl

jupyter_black.load()

In [ ]:
SEASON, WEEK = get_current_nfl_week()
WEEK = 7
SEASON, WEEK

In [ ]:
# 1. Load picks from database
week_range = range(1, 9)

picks_db = PicksDatabase()

pick_df = pd.concat(
    [pd.DataFrame(picks_db.get_picks(season=SEASON, week=w)) for w in week_range]
).drop(columns=["id", "created_at", "updated_at", "spread"])
pick_df

In [ ]:
pick_df.picker.value_counts()

In [ ]:
pool_db = PoolSpreadsDatabase()
raw_spreads = []
for week in week_range:
    raw_spreads.extend(pool_db.get_pool_spreads(season=SEASON, week=week))

pool_spreads = pd.DataFrame(raw_spreads).drop(
    columns=["created_at", "id", "season", "week"]
)
pool_spreads.sort_values(by="game_id")

In [ ]:
# 2. Load game results from nfl_data_py

# Import schedules for the season
schedule_df = nfl.import_schedules([SEASON])

# Filter to the specific week and get relevant columns
games_df = schedule_df[schedule_df["week"].isin(set(week_range))][
    ["game_id", "away_team", "home_team", "away_score", "home_score"]
].copy()

games_df.head()

In [ ]:
# 3. Merge picks with game results
merged_df = pick_df.merge(games_df, on="game_id", how="left").merge(
    pool_spreads, on="game_id", how="left"
)

# Check for any picks without game results (shouldn't happen if week is complete)
missing_results = merged_df[merged_df["away_score"].isna()]
if len(missing_results) > 0:
    print(
        f"⚠️ Warning: {len(missing_results)} picks have no game results (games may not be completed)"
    )
    print(missing_results[["picker", "team_picked", "game_id"]])
else:
    print("✅ All picks have game results")

merged_df.head()

In [ ]:
spread_picks = merged_df.query(
    'pick_type != "underdog" and pick_type != "survivor"'
).copy()
spread_picks

In [ ]:
# 4. Grade each pick (did they pick the winner?)
def grade_pick(row):
    """
    Determine if a pick was correct.
    Returns 1 for win, 0 for loss, 0.5 for push, None for incomplete game

    Note: Spreads are always in reference to the away team.
    Example: If away team is +3, they can lose by up to 2 and still cover.
             If away team is -3, they need to win by more than 3 to cover.
    """
    if pd.isna(row["away_score"]) or pd.isna(row["home_score"]):
        return None  # Game not completed

    picked_team = row["team_picked"]
    spread = row["spread"]

    margin = row["away_score"] - row["home_score"]

    away_covers = margin + spread

    if away_covers == 0:
        return 0.5

    if picked_team == row["away_team"]:
        return int(away_covers > 0)
    else:
        return int(away_covers < 0)


spread_picks["result"] = spread_picks.apply(grade_pick, axis=1)

# Add weight column (2 for best_bet, 1 for regular)
spread_picks["weight"] = spread_picks.apply(
    lambda row: 2 if row["pick_type"] == "best_bet" else 1, axis=1
)

spread_picks["weighted_result"] = spread_picks.apply(
    lambda row: 2 * row["result"] if row["pick_type"] == "best_bet" else row["result"],
    axis=1,
)
spread_picks

In [ ]:
spread_picks[(spread_picks["picker"] == "bModel")]

In [ ]:
filtered_picks = spread_picks[spread_picks["week"] > 0]

results = filtered_picks.groupby("picker").agg(
    {"game_id": "count", "result": "mean", "weighted_result": "sum", "weight": "sum"}
)

# Calculate proper weighted average: sum(weighted_result) / sum(weight)
results["weighted_avg"] = results["weighted_result"] / results["weight"]

results = results[["game_id", "result", "weighted_avg"]].rename(
    columns={
        "game_id": "picks",
        "result": "win_rate",
        "weighted_avg": "weighted_win_rate",
    }
)

results.sort_values("weighted_win_rate", ascending=False).round(3)

In [ ]:
filtered_picks = spread_picks[spread_picks["week"] > 4]

results = filtered_picks.groupby("picker").agg(
    {"game_id": "count", "result": "mean", "weighted_result": "sum", "weight": "sum"}
)

# Calculate proper weighted average: sum(weighted_result) / sum(weight)
results["weighted_avg"] = results["weighted_result"] / results["weight"]

results = results[["game_id", "result", "weighted_avg"]].rename(
    columns={
        "game_id": "picks",
        "result": "win_rate",
        "weighted_avg": "weighted_win_rate",
    }
)

results.sort_values("weighted_win_rate", ascending=False).round(3)